In [1]:
import os

assert os.environ['CONDA_DEFAULT_ENV']=='ttv'

## TOIs

In [2]:
import sys
sys.path.insert(0, '../code')
from utils import get_tois, get_ctois

In [3]:
df = get_tois()
df.tail()

Loaded:  ../data/TOIs.csv


,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Radius (R_Sun) err,Stellar Metallicity,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments
5299,428867906,6060.01,NaN,3,4,3,3,1,4,4,...,0.21,0.086,0.070,1.678,0.294829,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:44,low SNR; two stars in pixel
5300,239823606,6061.01,NaN,2,4,2,2,1,4,4,...,0.05,NaN,NaN,1.000,0.127632,"19,43,44,45,59",2023-02-08,2023-02-08,2023-02-10 12:02:44,two stars in pixel; low SNR
5301,122629761,6062.01,NaN,2,4,2,2,1,4,4,...,0.05,NaN,NaN,1.084,0.146070,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:43,variable host; low SNR
5302,67031480,6063.01,NaN,3,4,3,3,1,4,4,...,NaN,NaN,NaN,NaN,NaN,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:44,likely EB; a/Rs~2; host is member of M38 cluster
5303,428767018,6064.01,NaN,3,4,3,3,1,4,4,...,0.14,0.133,0.012,1.464,0.245446,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:43,NaN


In [4]:
tics = df['TIC ID'].unique()
len(tics)

5096

In [5]:
len(df['TOI'].unique())

5304

In [6]:
nsys = df['TIC ID'].value_counts()
nsys

367630162    5
425997655    5
150151262    4
142276270    4
260647166    4
            ..
59842822     1
124491883    1
143282930    1
147430118    1
428767018    1
Name: TIC ID, Length: 5096, dtype: int64

In [7]:
tics_multis = list(nsys[nsys>1].index)
print(len(tics_multis))

163


## TOI 1136 (Dai+2023)
First order MMR: 3:2, 2:1, 3:2, 7:5, and 3:2

In [8]:
col = 'Period (days)'
d = df[df['TIC ID']==142276270].sort_values(by=col)
d[col]

830     6.258792
829    12.518728
832    18.804756
831    26.317954
Name: Period (days), dtype: float64

In [9]:
# add two outer planets
periods = [4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]
periods

[4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]

https://arxiv.org/abs/1208.3312

$$
\frac{p'}{p} \sim \frac{j}{j-1}
$$

proximity to resonance:
$$
\Delta = \frac{p'}{p} \frac{j-1}{j} - 1
$$

In [10]:
import numpy as np
from itertools import combinations

def get_orbit_pairs(N=10, order=1):
    cs = list(combinations(np.arange(1, N), 2))
    cs = [(i,j) for i,j in cs if abs(i-j)==order]
    return cs

def get_resonant_pairs(periods, order=1, tol=0.01):
    """based on period commensurability"""
    periods = sorted(periods)
    Nplanets = len(periods) 
    
    resonant = []
    for n in range(Nplanets-1):
        Pout = periods[n+1]
        Pin = periods[n]
        for i,j in get_orbit_pairs(10, order=order):            
            delta = abs((Pout/Pin)*(i/j) - 1)
            if delta<=tol:
                text = f"{j}:{i} | P=({Pout:.2f},{Pin:.2f}) n=({n+1},{n+2}) (delta={delta*100:.2f}%)"
                resonant.append(text)
                break
    return resonant

In [11]:
get_orbit_pairs(10, order=1)

[(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]

In [12]:
resonances_1st_order = get_resonant_pairs(periods, order=1, tol=0.01)
resonances_1st_order

['3:2 | P=(6.26,4.17) n=(1,2) (delta=0.03%)',
 '2:1 | P=(12.52,6.26) n=(2,3) (delta=0.04%)',
 '3:2 | P=(18.80,12.52) n=(3,4) (delta=0.11%)',
 '3:2 | P=(39.54,26.32) n=(5,6) (delta=0.16%)']

In [13]:
resonances_2nd_order = get_resonant_pairs(periods, order=2, tol=0.01)
resonances_2nd_order

['6:4 | P=(6.26,4.17) n=(1,2) (delta=0.03%)',
 '4:2 | P=(12.52,6.26) n=(2,3) (delta=0.04%)',
 '6:4 | P=(18.80,12.52) n=(3,4) (delta=0.11%)',
 '7:5 | P=(26.32,18.80) n=(4,5) (delta=0.01%)',
 '6:4 | P=(39.54,26.32) n=(5,6) (delta=0.16%)']

## All TOIs

In [29]:
ttv_1st_order = []       # commensurability in 1 planet pair 
ttv_2nd_order = []
ttv_chain = []           # more than 1 planet pair
for tic in tics_multis:
    dd = df[df['TIC ID']==tic].squeeze()
    periods = dd["Period (days)"].values
    Nplanets = len(periods)
    toi = str(dd.iloc[0].TOI).split('.')[0] 
    
    resonances_1st_order = get_resonant_pairs(periods, order=1, tol=0.01);
    resonances_2nd_order = get_resonant_pairs(periods, order=2, tol=0.01);
    msg = f"TOI {toi} | TIC {tic} (nplanets={Nplanets}):<br>\n"
    show = False
    if len(resonances_1st_order)>=1:
        msg += f"First Order: {' '.join(resonances_1st_order)}<br>\n"
        ttv_1st_order.append(tic)
        if len(resonances_1st_order)>=2:
            ttv_chain.append(tic)
        show = True
    if len(resonances_2nd_order)>=1:
        msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
        ttv_2nd_order.append(tic)
        show = True
    if show:
        print(msg)

TOI 2104 | TIC 367630162 (nplanets=5):<br>
Second Order: 5:3 | P=(3.27,1.98) n=(1,2) (delta=0.85%)

TOI 174 | TIC 425997655 (nplanets=5):<br>
First Order: 2:1 | P=(7.91,3.98) n=(1,2) (delta=0.57%)<br>
Second Order: 4:2 | P=(7.91,3.98) n=(1,2) (delta=0.57%)

TOI 1136 | TIC 142276270 (nplanets=4):<br>
First Order: 2:1 | P=(12.52,6.26) n=(1,2) (delta=0.01%) 3:2 | P=(18.80,12.52) n=(2,3) (delta=0.14%)<br>
Second Order: 4:2 | P=(12.52,6.26) n=(1,2) (delta=0.01%) 6:4 | P=(18.80,12.52) n=(2,3) (delta=0.14%) 7:5 | P=(26.32,18.80) n=(3,4) (delta=0.03%)

TOI 700 | TIC 150428135 (nplanets=4):<br>
First Order: 4:3 | P=(37.42,27.81) n=(3,4) (delta=0.93%)<br>
Second Order: 8:6 | P=(37.42,27.81) n=(3,4) (delta=0.93%)

TOI 282 | TIC 29781292 (nplanets=4):<br>
First Order: 3:2 | P=(84.26,56.01) n=(3,4) (delta=0.30%)<br>
Second Order: 6:4 | P=(84.26,56.01) n=(3,4) (delta=0.30%)

TOI 1339 | TIC 269701147 (nplanets=3):<br>
First Order: 4:3 | P=(38.35,28.58) n=(2,3) (delta=0.64%)<br>
Second Order: 8:6 | P=

/tmp/ipykernel_12514/4028263970.py:19: RuntimeWarning: invalid value encountered in scalar divide
  delta = abs((Pout/Pin)*(i/j) - 1)
/tmp/ipykernel_12514/4028263970.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  delta = abs((Pout/Pin)*(i/j) - 1)


In [15]:
len(ttv_1st_order), len(ttv_2nd_order)

(18, 22)

In [16]:
import pandas as pd

s1 = pd.DataFrame(ttv_1st_order, columns=['tic'])
s1['1st order'] = True
s1.head()

,tic,1st order
0,425997655,True
1,142276270,True
2,150428135,True
3,29781292,True
4,269701147,True


In [17]:
s2 = pd.DataFrame(ttv_2nd_order, columns=['tic'])
s2['2nd order'] = True
s2.head()

,tic,2nd order
0,367630162,True
1,425997655,True
2,142276270,True
3,150428135,True
4,29781292,True


In [27]:
res = pd.merge(s1, s2, how='outer')
res

,tic,1st order,2nd order
0,425997655,True,True
1,142276270,True,True
2,150428135,True,True
3,29781292,True,True
4,269701147,True,True
5,307210830,True,True
6,54962195,True,True
7,318022259,True,True
8,259377017,True,True
9,441798995,True,True


In [18]:
res.to_csv('../data/tics_resonance_from_toi.csv')

In [19]:
np.savetxt('../data/tics_resonance_from_toi.txt', res.tic.tolist(), fmt="%d")
np.savetxt('../data/tics_resonance_from_toi_1st_order.txt', ttv_1st_order, fmt="%d")
np.savetxt('../data/tics_resonance_from_toi_2nd_order.txt', ttv_2nd_order, fmt="%d")

## CTOIS

In [20]:
df2 = get_ctois()
df2.head()

,TIC ID,CTOI,Promoted to TOI,MASTER,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Eff Temp (K) err,Stellar log(g) (cm/s^2),Stellar log(g) (cm/s^2) err,Stellar Radius (R_Sun),Stellar Radius (R_Sun) err,CTOI lastmod,User,Group,Tag,Notes
0,17361,17361.01,3127.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,121.637,4.25352,0.078445,1.25819,0.062226,2020-08-10 07:20:18,montalto,NaN,29419,"new CTOI, Montalto et al. (2020)"
1,160363,160363.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,129.081,4.15460,0.093692,1.66309,0.075980,2021-02-11 04:13:20,steuer,NaN,94179,Found with our in-house single transit finder
2,592503,592503.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.000,NaN,NaN,3.23242,NaN,2022-10-20 09:02:36,montalto,NaN,420769,"new CTOI, Montalto (2022)"
3,769986,769986.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,122.000,4.20725,NaN,1.04351,NaN,2022-10-20 09:04:55,montalto,NaN,421495,"new CTOI, Montalto (2022)"
4,991806,991806.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.480,NaN,NaN,3.63923,NaN,2022-10-20 09:02:36,montalto,NaN,420793,"new CTOI, Montalto (2022)"


In [21]:
nsys2 = df2['TIC ID'].value_counts()
nsys2

64837857      4
206544316     4
92349924      3
300903537     3
100504488     3
             ..
159059181     1
159098316     1
159159904     1
159185219     1
2050290974    1
Name: TIC ID, Length: 2707, dtype: int64

In [22]:
tics_multis2 = list(nsys2[nsys2>1].index)
print(len(tics_multis2))

136


In [23]:
ttv_1st_order2 = []       # commensurability in 1 planet pair 
ttv_2nd_order2 = []
ttv_chain2 = []           # more than 1 planet pair
for tic in tics_multis2:
    dd = df2[df2['TIC ID']==tic].squeeze()
    periods = dd["Period (days)"].values
    Nplanets = len(periods)
    
    resonances_1st_order = get_resonant_pairs(periods, order=1, tol=0.01);
    resonances_2nd_order = get_resonant_pairs(periods, order=2, tol=0.01);
    msg = f"TIC {tic} (nplanets={Nplanets}): <br>\n"
    show = False
    if len(resonances_1st_order)>=1:
        msg += f"First Order: {' '.join(resonances_1st_order)}\n"
        ttv_1st_order2.append(tic)
        if len(resonances_1st_order)>=2:
            ttv_chain2.append(tic)
        show = True
    if len(resonances_2nd_order)>=1:
        msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
        ttv_2nd_order2.append(tic)
        show = True
    if show:
        print(msg)

TIC 204497617 (nplanets=2): <br>
First Order: 2:1 | P=(3.18,1.59) n=(1,2) (delta=0.02%)
Second Order: 4:2 | P=(3.18,1.59) n=(1,2) (delta=0.02%)

TIC 29478974 (nplanets=2): <br>
First Order: 2:1 | P=(24.90,12.46) n=(1,2) (delta=0.06%)
Second Order: 4:2 | P=(24.90,12.46) n=(1,2) (delta=0.06%)

TIC 207080350 (nplanets=2): <br>
First Order: 2:1 | P=(18.38,9.11) n=(1,2) (delta=0.94%)
Second Order: 4:2 | P=(18.38,9.11) n=(1,2) (delta=0.94%)

TIC 347332255 (nplanets=2): <br>
First Order: 3:2 | P=(13.67,9.11) n=(1,2) (delta=0.02%)
Second Order: 6:4 | P=(13.67,9.11) n=(1,2) (delta=0.02%)

TIC 253922296 (nplanets=2): <br>
Second Order: 3:1 | P=(9.43,3.14) n=(1,2) (delta=0.03%)



/tmp/ipykernel_12514/4028263970.py:19: RuntimeWarning: invalid value encountered in scalar divide
  delta = abs((Pout/Pin)*(i/j) - 1)
/tmp/ipykernel_12514/4028263970.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  delta = abs((Pout/Pin)*(i/j) - 1)


In [24]:
s1 = pd.DataFrame(ttv_1st_order2, columns=['tic'])
s1['1st order'] = True
s2 = pd.DataFrame(ttv_2nd_order2, columns=['tic'])
s2['2nd order'] = True
res2 = pd.merge(s1, s2, how='outer')
res2

,tic,1st order,2nd order
0,204497617,True,True
1,29478974,True,True
2,207080350,True,True
3,347332255,True,True
4,253922296,NaN,True


In [25]:
res2.to_csv('../data/tics_resonance_from_ctoi.csv')

In [26]:
np.savetxt('../data/tics_resonance_from_ctoi.txt', res2.tic.tolist(), fmt="%d")
np.savetxt('../data/tics_resonance_from_ctoi_1st_order.txt', ttv_1st_order2, fmt="%d")
np.savetxt('../data/tics_resonance_from_ctoi_2nd_order.txt', ttv_2nd_order2, fmt="%d")

## Transit timing database
* https://ui.adsabs.harvard.edu/abs/2022ApJS..259...62I/abstract

In [103]:
#https://github.com/transit-timing/tt/blob/master/3_database/table4.csv
url = "https://raw.githubusercontent.com/transit-timing/tt/master/3_database/table4.csv"
df2 = pd.read_csv(url)
df2

,System,Orbit number,T_mid,Uncertainty (days),Time System,#,Reference
0,CoRoT-01,-1412,2.454138e+06,0.000470,BJD,1,2009A&A...506..369B
1,CoRoT-01,-1411,2.454140e+06,0.000590,BJD,1,2009A&A...506..369B
2,CoRoT-01,-1410,2.454141e+06,0.000620,BJD,1,2009A&A...506..369B
3,CoRoT-01,-1409,2.454143e+06,0.000390,BJD,1,2009A&A...506..369B
4,CoRoT-01,-1408,2.454144e+06,0.001630,BJD,1,2009A&A...506..369B
...,...,...,...,...,...,...,...
8662,XO-7,214,2.459407e+06,0.000510,BJD_TDB,1,This work
8663,XO-7,215,2.459410e+06,0.000467,BJD_TDB,1,This work
8664,XO-7,216,2.459413e+06,0.000508,BJD_TDB,1,This work
8665,XO-7,217,2.459415e+06,0.000517,BJD_TDB,1,This work
